# Byline 적재

In [1]:
from pathlib import Path
import os

os.chdir(Path(os.getcwd()).parent.parent)

In [14]:
from os.path import join

import json

import pandas as pd
import numpy as np

In [15]:
root_path = os.getcwd()
data_path = join(os.getcwd(), "data")
pre_path = join(data_path, "preprocessing")
bulk_path = join(data_path, "bulk")

## Load bylines.csv

In [16]:
target_name = "byline"
target_file_name = "bylines.csv"

In [17]:
target_df = pd.read_csv(join(pre_path, target_file_name))

In [18]:
target_df.head()

,이름,정규화
0,(中）/[번역]강지혜\n[번역]강지혜,강지혜
1,(강원)강대웅·위준휘\n위준휘,강대웅||위준휘
2,(경주) 최주호\n최주호,최주호
3,(과천) 박재천\n박재천,박재천
4,(광주)박승호\n박승호,박승호


In [19]:
target_df = target_df.rename(columns={"이름": "origin", "정규화": "names"})

In [20]:
target_df.head()

,origin,names
0,(中）/[번역]강지혜\n[번역]강지혜,강지혜
1,(강원)강대웅·위준휘\n위준휘,강대웅||위준휘
2,(경주) 최주호\n최주호,최주호
3,(과천) 박재천\n박재천,박재천
4,(광주)박승호\n박승호,박승호


## Bulk

In [21]:
action_str = {
    "index": {
        "_index": target_name
    }
}

action = json.dumps(action_str)

In [31]:
def get_row_dict(row):
    origin, raw_name = row["origin"], row["names"]

    if isinstance(raw_name, float):
        return {"origin": origin}

    names = raw_name.split("||")

    if len(names) == 1:
        return {"origin": origin, "names": names[0]}

    return {"origin": origin, "names": names}

In [32]:
bulk_data = ""

count = 0
for index, row in target_df.iterrows():
    row_dict = get_row_dict(row)
    row_bulk = json.dumps(row_dict, ensure_ascii=False)
    bulk_data += f"{action}\n{row_bulk}\n"

In [33]:
with open(join(bulk_path, f"{target_name}.json"), "w", encoding="utf-8") as json_file:
    json_file.write(bulk_data)